# CS440 MP1

In [168]:
import urllib2
from collections import deque

## Maze Class and Maze Solver Class

In [169]:
class Maze:
    '''
    Creates a data structure to represent the maze problem.
    '''
    def __init__(self, txtFile):
        i = 0 #row index
        data = urllib2.urlopen(txtFile)
        self.maze = []
        self.goals = []
        self.explored = []
        for line in data:
            j = 0 #column index
            self.maze.append([])
            for char in line:
                #find goal state or goal states
                if char == '.':
                    self.goals.append((i,j))
                #find initial state
                elif char == 'P':
                    self.initialState = (i,j)
                if char == 'P' or char == '.' or char == '%' or char == ' ': 
                    self.maze[i].append(char)
                    j+=1
            i+=1
        self.height = i
        self.width = j
        
    def findChildren(self, node):
        children = []
        x = node[1]
        y = node[0]
        if x-1 >= 0:
            if self.maze[y][x-1] != '%':
                children.append((y,x-1))
                
        if y-1 >= 0:
            if self.maze[y-1][x] != '%':
                children.append((y-1,x))
            
        if x+1 < self.width:
            if self.maze[y][x+1] != '%':
                children.append((y,x+1))

        if y+1 < self.height:
            if self.maze[y+1][x] != '%':
                children.append((y+1,x))
        return children
    
    def printMazeWithPath(self, path):
        for row in range(self.height):
            for col in range(self.width):
                if (row,col) in path:
                    print ".",
                else:
                    print self.maze[row][col],
            print " "
        

class MazeSearch:
    '''
    Base class outlining functions that the inherited search classes should contain.
    Should not be created or implemented.
    '''
    def __init__(self, maze):
        raise NotImplementedError
    
    def pathFinder(self):
        self.addNodeToFrontier(self.maze.initialState) 
        while not self.emptyFrontier() :
            self.numNodesVisited+=1
            node = self.chooseNodeFromFrontier()
            if self.goalState(node): #goal state is reached
                return self.backTrace(node)
            for child in self.maze.findChildren(node):
                if not self.duplicateDetection(child):
                    self.addNodeToFrontier(child)
                    self.parent[child[0]][child[1]] = node
                    self.explored.append(node)
        return -1
    
    def backTrace(self, node):
        path = []
        curNode = node
        while curNode != self.maze.initialState:
            path.append(curNode)
            parent = self.parent[curNode[0]][curNode[1]]
            curNode = parent
        return path
    
    def duplicateDetection(self, node):
        '''
        Implement strategy for detecting duplicate states.
        '''
        raise NotImplementedError
    
    def addNodeToFrontier(self, node):
        '''
        Adds node to the frontier. The data structure representing a frontier might differ
        for different search strategies.
        input: void
        return: void
        '''
        raise NotImplementedError
        
    def chooseNodeFromFrontier(self):
        raise NotImplementedError
        
    def emptyFrontier(self):
        raise NotImplementedError
        
    def goalState(self, node):
        raise NotImplementedError

##Depth First Search 

In [167]:
class DFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = []
        self.maze = maze
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        #if node in self.frontier:
        
    def addNodeToFrontier(self, node):
        self.frontier.append(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.pop()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals

In [170]:
medMaze = Maze("http://slazebni.cs.illinois.edu/fall15/assignment1/mediumMaze.txt")
DFS = DFSMazeSearch(medMaze)
path = DFS.pathFinder()
medMaze.printMazeWithPath(path)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

##Breadth First Search

In [165]:
class BFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = deque([])
        self.maze = maze
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        if node in self.frontier:
            return True
        
    def addNodeToFrontier(self, node):
        self.frontier.append(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.popleft()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals

In [166]:
medMaze = Maze("http://slazebni.cs.illinois.edu/fall15/assignment1/mediumMaze.txt")
BFS = BFSMazeSearch(medMaze)
path = BFS.pathFinder()
medMaze.printMazeWithPath(path)

% % % % % % % % % % % % % % % % % % % % % % %  
% . %       %       %       %       %   %   %  
% .     %       % %   %   % % % %   % %     %  
% . %       %   %   %   %               %   %  
% . % %   % % %           % %   % %   % % % %  
% . %   %       %   %   %   %   %           %  
% . % % %   %   % % %     %         % %   % %  
% . %   % . . .         %   %   %       %   %  
% . . . . . % . %   % % %       %   %       %  
%   %   %   % . . . . . %   %   %       %   %  
%       % % %   % % % . %   %       %   % % %  
%   %           %   % . . . . . %           %  
% %   %   % %         % % % % . %   % %   % %  
%   %   %       %   %         . . .     %   %  
%       % %   % %   % %   % % % % . % %     %  
%   %   %           %           % . %   %   %  
% % % %   % %   % %     % %   %   .   %   % %  
%   %   %           %       %   % . %   %   %  
%   % %   %   % % % % %   %       . . .     %  
%   %               %       %   %   % . %   %  
%       % %   % %   % % %   %       % . 

In [ ]:
class GBFSMazeSearch(MazeSearch):
    def __init__(self, maze):
        self.frontier = deque([])
        self.maze = maze
        self.parent = [[(0,0) for j in range(maze.width)] for i in range(maze.height)]
        self.numNodesVisited = 0
        self.explored = []
    
    def duplicateDetection(self, node):
        if node in self.explored:
            return True
        if node in self.frontier:
            return True
        
    def addNodeToFrontier(self, node):
        self.frontier.append(node)
        
    def chooseNodeFromFrontier(self):
        return self.frontier.popleft()
    
    def emptyFrontier(self):
        return not self.frontier
    
    def goalState(self, node):
        return node in self.maze.goals